In [104]:
pip install tensorflow



   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.8/390.3 MB 2.8 MB/s eta 0:02:20
   ---------------------------------------- 1.3/390.3 MB 3.0 MB/s eta 0:02:08
   ---------------------------------------- 2.1/390.3 MB 3.1 MB/s eta 0:02:06
   ---------------------------------------- 2.9/390.3 MB 3.2 MB/s eta 0:02:01
   ---------------------------------------- 3.1/390.3 MB 2.9 MB/s eta 0:02:15
   ---------------------------------------- 3.7/390.3 MB 2.9 MB/s eta 0:02:13
   ---------------------------------------- 4.5/390.3 MB 2.9 MB/s eta 0:02:11
    --------------------------------------- 5.0/390.3 MB 2.9 MB/s eta 0:02:12
    --------------------------------------- 5.2/390.3 MB 2.9 MB/s eta 0:02:13
    --------------------------------------- 5.8/390.3 MB 2.7 MB/s eta 0:02:21
    --------------------------------------- 6.3/390.3 MB 2.7 MB/s eta 0:02:25

In [4]:
import pandas as pd
import numpy as np
import transformers
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

In [6]:
power=pd.read_csv("Plant_1_Generation_Data.csv")
sensor=pd.read_csv("Plant_1_Weather_Sensor_Data.csv")
power=power.dropna()
sensor=sensor.dropna()
sensor.head()



,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0


In [8]:
power.isnull().sum()

DATE_TIME      0
PLANT_ID       0
SOURCE_KEY     0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
dtype: int64

In [10]:
power['DATE_TIME'] = pd.to_datetime(power['DATE_TIME'],format = '%d-%m-%Y %H:%M')


In [12]:
sensor['DATE_TIME'] = pd.to_datetime(sensor['DATE_TIME'],format = '%Y-%m-%d %H:%M:%S')


In [14]:
sensor.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0


In [16]:
power = pd.merge(power.drop(columns = ['PLANT_ID']), sensor.drop(columns = ['PLANT_ID', 'SOURCE_KEY']), on='DATE_TIME')

In [18]:
power['DATE_TIME'] = pd.to_datetime(power['DATE_TIME'])
power['DATE'] = power['DATE_TIME'].dt.date
power['TIME'] = power['DATE_TIME'].dt.time
power['HOURS'] = power['DATE_TIME'].dt.hour
power['MINUTES'] = power['DATE_TIME'].dt.minute
power['MINUTES_PASS'] = power['HOURS'] * 60 + power['MINUTES']

In [20]:
power.head()

,DATE_TIME,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DATE,TIME,HOURS,MINUTES,MINUTES_PASS
0,2020-05-15,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
1,2020-05-15,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
2,2020-05-15,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
3,2020-05-15,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
4,2020-05-15,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0


In [27]:
X=power[['DATE_TIME','SOURCE_KEY','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','DATE','TIME','HOURS','MINUTES','MINUTES_PASS']]

In [29]:
X.head()

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DATE,TIME,HOURS,MINUTES,MINUTES_PASS
0,2020-05-15,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
1,2020-05-15,1IF53ai7Xc0U56Y,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
2,2020-05-15,3PZuoBAID5Wc2HD,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
3,2020-05-15,7JYdWkrLSPkdwr4,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0
4,2020-05-15,McdE0feGgRqW7Ca,25.184316,22.857507,0.0,2020-05-15,00:00:00,0,0,0


In [31]:
Y=power[["AC_POWER"]]

In [2]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

NameError: name 'train_test_split' is not defined